In [1]:
pip install opencv-python

In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyTelegramBotAPI

  Using cached pyTelegramBotAPI-4.14.1.tar.gz (244 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.6-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2023.11.17-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached certifi-2023.11.17-py3-none-any.whl (162 kB)
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/100.4 kB ? eta -:--:--
   --------------- ----------------------- 41.0/100.4 kB 653.6 kB/s eta 0:00:01
   --------------- ---

In [13]:
import cv2
import numpy as np 
import telebot
from telebot import types 


bot = telebot.TeleBot("6968701267:AAGqyb-ZDkJ1efwGjc6c-i4X4ldK0pWzS3c")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):

    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file= bot.download_file(file_info.file_path)

    with open("photo.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)


    img = cv2.imread("photo.jpg")


    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))

    for(x,y,w,h) in faces:
        face_roi = gray [y:y + h, x:x + w]
        edges = cv2.Canny(face_roi, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)

        if lines is not None:
            cv2.rectangle(img, (x,y),(x+w, y+h), (0,255,0),2)

            cv2.imwrite("result.jpg", img)

            with open ("result.jpg", 'rb') as result_file:
                bot.send_photo(message.chat.id, result_file, "نم التقاط صورة شخص يحمل الجوال")
            
                return
    bot.reply_to(message, "لم يتم العثور عبى شخص يحمل هاتف")


if __name__== "__main__":
    bot.polling(none_stop=True)            
        



In [12]:
import cv2
import numpy as np
import telebot
from telebot import types

# قم بتبديل 'YOUR_TELEGRAM_BOT_TOKEN' برمز البوت الخاص بك على تيليجرام الفعلي
TOKEN = 'TOKEN'
bot = telebot.TeleBot("6968701267:AAGqyb-ZDkJ1efwGjc6c-i4X4ldK0pWzS3c")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    # قم بتنزيل الصورة
    file_info = bot.get_file(message.photo[-1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)

    # احفظ الصورة على القرص المحلي
    with open("photo.jpg", 'wb') as new_file:
        new_file.write(downloaded_file)

    # قم بتحميل الصورة باستخدام OpenCV
    image = cv2.imread("photo.jpg")

    # قم بتحويل الصورة إلى اللون الرمادي
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # قم بتحميل مصنف كاسكيد الذي يُستخدم لاكتشاف الوجوه مسبقًا
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # اكتشف الوجوه في الصورة
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    # تحقق مما إذا كان يوجد شخص يحمل جوال في كل وجه تم اكتشافه
    for (x, y, w, h) in faces:
        face_roi = gray[y:y + h, x:x + w]
        edges = cv2.Canny(face_roi, 50, 150)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=30, maxLineGap=10)

        if lines is not None:
            # قم برسم مستطيل أخضر حول الجوال المكتشف
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # احفظ الصورة مع المستطيل الأخضر
            cv2.imwrite("result.jpg", image)

            # أرسل الصورة المعدلة إلى المستخدم
            with open("result.jpg", 'rb') as result_file:
                bot.send_photo(message.chat.id, result_file, "تم إلتقاط صورة لشخص يحمل الجوال")

            return

    # إذا لم يتم العثور على شخص يحمل جوال
    bot.reply_to(message, "لم يتم العثور على شخص يحمل جوال في الصورة.")

if __name__ == "__main__":
    bot.polling(none_stop=True)
